In [8]:
%matplotlib widget

In [9]:
import nmrProblemGUI

In [10]:
ipwidgetsGUI = nmrProblemGUI.ipywidgetsDisplay()

first if statement


In [11]:
ipwidgetsGUI

ipywidgetsDisplay(children=(VBox(children=(Accordion(children=(HBox(children=(FileUpload(value={}, description…

In [ ]:
ipwidgetsGUI.nmrproblem.udic[0]['info'].keys()

In [ ]:
nmrproblem = ipwidgetsGUI.nmrproblem
df = nmrproblem.df
iprobs = nmrproblem.iprobs

In [ ]:
iprobs['C4']

In [ ]:
C13df_orig.loc[iprobs['C4']]

In [ ]:
C13df_orig[['sF_latex_coloured', 'minCS', 'maxCS']].tail()

In [ ]:
H1df, C13df, probDistFunctions = calcProbDistFunctions(nmrproblem, H1df_orig, C13df_orig)

In [ ]:
ccc = probDistFunctions.loc[iprobs['C2'], 'C2'].head()
ccc

In [ ]:
hhh =probDistFunctions.loc[iprobs[df.loc['hsqc', 'C2'][0]], df.loc['hsqc', 'C2'][0]].head()
hhh

In [ ]:
df.loc['hsqc', 'C2'][0]

In [ ]:
hset = set(H1df.loc[iprobs['H1']].substituent.head())
hset

In [ ]:
cset = set(C13df.loc[iprobs['C2']].substituent.head())
cset

In [ ]:
cset.intersection(hset)

In [ ]:
for sub in cset.intersection(hset):
    h1 = list(H1df.loc[iprobs['H1']].head()[H1df.loc[iprobs['H1']].head().substituent == sub]['H1'])
    c1 = list(C13df.loc[iprobs['C2']].head()[C13df.loc[iprobs['C2']].head().substituent == sub]['C2'])
    for h in h1:
        for c in c1:
#             print(sub, -np.log10(h), -np.log10(c), -np.log10(h*c))
            print(sub, h, c, h*c)

In [ ]:
C13df.loc[iprobs['C2']].head()[C13df.loc[iprobs['C2']].head().substituent == 'amide']

In [ ]:
H1df.loc[iprobs['H1']].head()[H1df.loc[iprobs['H1']].head().substituent == 'amide']

In [ ]:
H1df.columns

In [ ]:
H1df_orig.loc[60]

In [ ]:
pd.DataFrame( np.outer(hhh, ccc), columns = list(C13df_orig.loc[nmrproblem.iprobs['C2']].sF_latex_coloured),
             index= list(H1df_orig.loc[nmrproblem.iprobs['H1']].sF_latex_coloured))

In [ ]:
np.outer([1,2,3],[1,2])

In [ ]:

M_substituents = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Fr', 'Si', 'Al', 'B']

elements = nmrproblem.elements
proton_atoms = nmrproblem.protonAtoms
carbon_atoms = nmrproblem.carbonAtoms

# reduce by DBE
DBE = int(nmrproblem.dbe)
H1df = H1df[H1df['DBE'] <= DBE]
C13df = C13df[C13df['DBE'] <= DBE]

freeProtons = elements['H'] - nmrproblem.df.loc['integral',proton_atoms].sum()

# If DBE equals 1, is it due to double bond oxygen, Nitrogen, Halogen or alkene ?
carbonDBE=False
oxygenDBE=False
halogenDBE=False
nitrogenDBE=False
hydroxylsPresent = False
numHydroxyls = 0
freeOxygens=0
elements = nmrproblem.elements
freeProtons = elements['H'] - nmrproblem.df.loc['integral',proton_atoms].sum()

# Find out if there are hydroxyls
freeProtons = nmrproblem.elements['H'] - nmrproblem.df.loc['integral',proton_atoms].sum()
if DBE == 0:
    if freeProtons > 0 and 'O' in nmrproblem.elements:
        hydroxylsPresent = True
        numHydroxyls=freeProtons
        freeOxygens  = nmrproblem.elements['O'] - freeProtons

# Remove alkenes from table if DBE due to oxygen
#
# oxygenDBE = True
# print(H1df.shape, C13df.shape)
if ((DBE == 1) or (DBE>=5 and elements['C'] > 6)) and (oxygenDBE):
    # remove alkene groups

    H1df = H1df[H1df.groupName != 'alkene']
    H1df = H1df[H1df.substituent != 'alkene']
    C13df = C13df[C13df.groupName != 'alkene']
    C13df = C13df[C13df.substituent != 'alkene']

# remove choices where atoms not present, N, O, S, Halogens, metals
### protons

for e in ['O','S','N']:
    if e not in elements:
        H1df=H1df[H1df[e]==0]
    else:
        H1df=H1df[H1df[e]<=elements[e]]

no_halides = True

halide_elements =[e for e in elements.keys() if e in ['F','Cl','Br', 'I']]
if len(halide_elements)>0:
    no_halides = False

if no_halides:
    H1df = H1df[H1df.substituent != 'halide']

# remove metals from dataframe in none found
no_Ms = True
M_elements =[e for e in elements.keys() if e in M_substituents]
if len(M_elements)>0:
    no_Ms = False

if no_Ms:
    H1df = H1df[H1df.substituent != 'metal']

### carbons

for e in ['O','S','N','F', 'Cl', 'Br', 'I']:
    if e not in elements:
        C13df = C13df[C13df[e] == 0]
    else:
        C13df = C13df[C13df[e]<=elements[e]]

# Now to reduce the choice further by using integrals, multiplicity.
# This should now be done for individual peaks and the results kept separately

# start with carbons
#
iprobs = {}
for c in carbon_atoms:
    kept_i = []
    for i in C13df.index:
        if int(nmrproblem.df.loc['C13 hyb', c]) in C13df.loc[i,'attached_protons']:
            kept_i.append(i)
    iprobs[c]=kept_i



# if attached protons / hybr == 3 remove other guesses other than C13 methyl

for c in carbon_atoms:
    if nmrproblem.df.loc['C13 hyb', c] == 3:
        kept_i = []
        for i in iprobs[c]:
            # if first pos in list is 3 then group in table is a methyl
            if C13df.loc[i,'attached_protons'][0] == 3:
                kept_i.append(i)
        iprobs[c]=kept_i

print(iprobs)

## now reduce H1 options based on CH2, CH3, CH1
## use the integral field in nmr properties table 

for h in proton_atoms:
    kept_i = []
    for i in H1df.index:
        if nmrproblem.df.loc['C13 hyb', h] in H1df.loc[i,'num_protons']:
            kept_i.append(i)
    iprobs[h]=kept_i
    
print(iprobs)

# check to see if proton is attached to a carbon
#
for h in proton_atoms:
    kept_i = []
    for i in iprobs[h]:
        if (int(nmrproblem.df.loc['C13 hyb', h]) > 0) and (H1df.loc[i,'carbon_attached'] == 1):
            kept_i.append(i)
        elif (int(nmrproblem.df.loc['C13 hyb', h]) == 0) and (H1df.loc[i,'carbon_attached'] == 0):
            kept_i.append(i)
    iprobs[h]=kept_i

# sort the candidates, highes first
#
for n, iii in iprobs.items():
    # print(n,iii)
#        sss = probDistFunctions.loc[iii, n].sort_values(ascending=False)
#        iprobs[n]=sss.index.tolist()
    if n in proton_atoms:
        sss = H1df.loc[iii, n].sort_values(ascending=False)
    elif n in carbon_atoms:
        sss = C13df.loc[iii, n].sort_values(ascending=False)
    iprobs[n]=sss.index.tolist()

iprobs


In [ ]:
createH1C13interactivePlot(widgets.Tab())

In [ ]:
ffff = {}

In [ ]:
ffff[0][]

In [ ]:
plotC13Distributions(ipwidgetsGUI, 4, 3)

In [ ]:
upload_problemdir  = ipywidgets.widgets.FileUpload(multiple=True, 
                                                        description="Open Existing Problem ",
                                                        description_tooltip="choose all files in problem directory",
                                                        layout = widgets.Layout(width='300px'))


problemNameL = widgets.Label(value="    Problem Name",
                            layout = widgets.Layout(width='100px'))

spacerL = widgets.Label(value="    ",
                            layout = widgets.Layout(width='50px'))

problemNameW =widgets.Text(value="Problem Name",
                           description="",
                          layout = widgets.Layout(width='150px'))


newproblemB = widgets.Button( description="Start New Problem")

prDirLayout = widgets.HBox([upload_problemdir,
                            spacerL,
                            problemNameL,
                            problemNameW,
                            spacerL,
                            newproblemB])

In [ ]:
prDirLayout

In [ ]:
www = widgets.Output()

In [ ]:
with www:
    print("Hello World")

In [ ]:
www

In [ ]:
"pr int".replace(" ", "")

In [ ]:
tmpdir = tempfile.TemporaryDirectory()
os.mkdir(os.path.join(tmpdir.name, "test"))
nmrproblem = NMRproblem2(os.path.join(tmpdir.name, "test"))

In [ ]:
print(dir(tmpdir))

In [ ]:
os.listdir(tmpdir.name)

In [ ]:
nmrproblem = NMRproblem2(os.path.join(tmpdir.name, "test"))

In [ ]:
nmrproblem.df